In [441]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [442]:
#Import data

customer_content = "customer_content.csv"
data_ticket = "data_ticket.csv"
working_hours = "working_hours.csv"
customer_feedback="customer_feedback.csv"

df_customer_content = pd.read_csv(data_ticket)
df_data_ticket = pd.read_csv(data_ticket)
df_working_hours = pd.read_csv(working_hours)
df_customer_feedback=pd.read_csv(customer_feedback)

# Part A

#### Q1. Calculate the total number of hours worked by employee per week, knowing that working time does not include break time?

In [443]:
df_working_hours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56783 entries, 0 to 56782
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   AUTO_ID      56783 non-null  int64 
 1   AGENT_ID     56783 non-null  object
 2   TYPE_ACTION  56783 non-null  object
 3   CREATE_DATE  56783 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


In [444]:
# 1. Convert CREATE_DATE (epoch ms) -> datetime Vietnam time (UTC+7)
df_working_hours["EVENT_TIME"] = (
    pd.to_datetime(df_working_hours["CREATE_DATE"], unit="ms", utc=True)
      .dt.tz_convert("Asia/Ho_Chi_Minh")
)

# 2. Sort theo AGENT_ID & thời gian (BẮT BUỘC)
df_working_hours = df_working_hours.sort_values(
    by=["AGENT_ID", "EVENT_TIME"]
)

# 3. Lấy thời điểm event kế tiếp cho mỗi AGENT_ID
df_working_hours["NEXT_EVENT_TIME"] = (
    df_working_hours
    .groupby("AGENT_ID")["EVENT_TIME"]
    .shift(-1)
)

# 4. Tính duration giữa current event và next event (giờ)
df_working_hours["DURATION_HOURS"] = (
    df_working_hours["NEXT_EVENT_TIME"]
    - df_working_hours["EVENT_TIME"]
).dt.total_seconds() / 3600

# 5. CHỈ giữ các current event được tính là thời gian làm việc
df_working = df_working_hours[
    df_working_hours["TYPE_ACTION"].isin(
        ["START_SHIFT", "START_WORKING"]
    )
].copy()


In [445]:
# Use descriptive statistics to identify outliers in working duration,
# caused by events that do not follow the expected business logic
df_working["DURATION_HOURS"].describe()

count    28455.000000
mean         2.941912
std         16.342318
min          0.000000
25%          0.046111
50%          0.423056
75%          1.355972
max       1060.113056
Name: DURATION_HOURS, dtype: float64

In [446]:
#The median (0.58 hours) and 75th percentile (1.36 hours) indicate that most working segments are relatively short.
#The maximum duration (1066 hours) is significantly higher than the upper quartiles.
#The large gap between the 75th percentile and the maximum value suggests the presence of extreme outliers in the data.
#Such long durations are unlikely to represent continuous working behavior and are more likely caused by logging anomalies (e.g. missing events).
#Solution: filter durations <= 8 hours due to business rules (1 working shift no longer 8 hours)

df_working = df_working[df_working["DURATION_HOURS"] <= 8]
df_working["YEAR"] = df_working["EVENT_TIME"].dt.isocalendar().year
df_working["WEEK"] = df_working["EVENT_TIME"].dt.isocalendar().week
df_working["WEEK_START_DATE"] = (
    df_working["EVENT_TIME"]
    - pd.to_timedelta(df_working["EVENT_TIME"].dt.weekday, unit="D")
).dt.date
weekly_working_hours = (
    df_working
    .groupby(
        ["AGENT_ID", "YEAR", "WEEK", "WEEK_START_DATE"],
        as_index=False
    )
    .agg(
        TOTAL_WORKED_HOURS=("DURATION_HOURS", "sum")
    )
)

In [448]:
# Weekly working hours per employee
weekly_working_hours.to_csv('weekly_working_hours.csv',index=False)

#### 2. Which employee has the most working hours?

In [189]:
#Top employee by total hours by data set
total_hours_by_employee = (
    df_working
    .groupby("AGENT_ID", as_index=False)
    .agg(
        TOTAL_WORKED_HOURS=("DURATION_HOURS", "sum")
    )
    .sort_values("TOTAL_WORKED_HOURS", ascending=False)
)

# Top employee
total_hours_by_employee.head(1)


,AGENT_ID,TOTAL_WORKED_HOURS
14,bpomb33,500.346667


In [190]:
#Top employee by total hours per month

df_working["MONTH"] = (
    df_working["EVENT_TIME"]
    .dt.month
)

monthly_hours_by_employee = (
    df_working
    .groupby(["AGENT_ID", "MONTH"], as_index=False)
    .agg(
        TOTAL_WORKED_HOURS=("DURATION_HOURS", "sum")
    )
    .sort_values(["MONTH", "TOTAL_WORKED_HOURS"], ascending=[True, False])
)
top_employee_each_month = (
    monthly_hours_by_employee
    .sort_values(["MONTH", "TOTAL_WORKED_HOURS"], ascending=[True, False])
    .groupby("MONTH")
    .head(1)
)

# Top employee
top_employee_each_month


,AGENT_ID,MONTH,TOTAL_WORKED_HOURS
39,bpomb33,6,268.786389
11,bpomb15,7,245.090556
38,bpomb32,8,28.810556


#### 3. Calculate the total number of tickets received per week?

In [191]:
df_data_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60675 entries, 0 to 60674
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             60675 non-null  object
 1   Agent_tiep_nhan  60658 non-null  object
 2   Level1           60651 non-null  object
 3   Total_ticket     60675 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.9+ MB


In [192]:
df_data_ticket['Agent_tiep_nhan'] = df_data_ticket['Agent_tiep_nhan'].fillna('XNA')
df_data_ticket['Level1'] = df_data_ticket['Level1'].fillna('XNA')

In [193]:
df_data_ticket=df_data_ticket.rename(columns={
                                     'Date ':'Date'
                                    })

In [194]:
df_data_ticket['Date']=pd.to_datetime(df_data_ticket['Date'])
df_data_ticket['Week']=df_data_ticket['Date'].dt.isocalendar().week
df_data_ticket['Month']=df_data_ticket['Date'].dt.month
df_data_ticket['Year']=df_data_ticket['Date'].dt.year
week_start = (
    df_data_ticket["Date"]
    - pd.to_timedelta(df_data_ticket["Date"].dt.weekday, unit="D")
)

# 3. (optional nhưng NÊN) ghi đè lại WEEK cho đồng bộ
df_data_ticket["WEEK"] = week_start.dt.isocalendar().week

# 4. tạo WEEK_RANGE
df_data_ticket["WEEK_RANGE"] = (
    week_start.dt.strftime("%Y-%m-%d")
    + " to "
    + (week_start + pd.Timedelta(days=6)).dt.strftime("%Y-%m-%d")
)

In [195]:
total_ticket_per_week=df_data_ticket.groupby(['Year','Month','Week','WEEK_RANGE']).agg({
'Total_ticket':'sum'
}).reset_index()

In [196]:
total_ticket_per_week

,Year,Month,Week,WEEK_RANGE,Total_ticket
0,2025,6,22,2025-05-26 to 2025-06-01,1940
1,2025,6,23,2025-06-02 to 2025-06-08,17998
2,2025,6,24,2025-06-09 to 2025-06-15,17311
3,2025,6,25,2025-06-16 to 2025-06-22,19055
4,2025,6,26,2025-06-23 to 2025-06-29,19422
5,2025,6,27,2025-06-30 to 2025-07-06,3084
6,2025,7,27,2025-06-30 to 2025-07-06,16871
7,2025,7,28,2025-07-07 to 2025-07-13,20585
8,2025,7,29,2025-07-14 to 2025-07-20,21457
9,2025,7,30,2025-07-21 to 2025-07-27,22374


#### 4.  Identify the most frequent issue reported by customers? 

In [450]:
issue_group_map = {

    # ===================== ACC | Account / Security =====================
    "Hỗ trợ tài khoản": "ACC",
    "Xác thực tài khoản": "ACC",
    "Xác thực tài khoản (eKYC)": "ACC",
    "Mở tài khoản": "ACC",
    "Tài khoản Account": "ACC",
    "Tài Khoản": "ACC",
    "Mật khẩu": "ACC",
    "Khóa Ví": "ACC",
    "Mở khóa Ví": "ACC",
    "Khóa tài khoản": "ACC",
    "Hạn mức Ví": "ACC",
    "Hạn mức Ví": "ACC",
    "Hạn mức 24h": "ACC",
    "Ví Trả Sau": "ACC",
    "Nguồn tiền hạn mức cao": "ACC",
    "Điểm tin cậy": "ACC",
    "Chính sách riêng tư": "ACC",
    "Liên kết": "ACC",
    "Liên kết tài khoản": "ACC",
    "Liên kết ngân hàng": "ACC",
    "Liên kết ngân hàng 1 chạm": "ACC",
    "Hủy liên kết": "ACC",
    "Mở chặn giao dịch": "ACC",
    "Xem số dư Ngân hàng": "ACC",

    # ===================== PAY | Transfer / Topup / Withdraw =====================
    "Nạp tiền": "PAY",
    "Ngân hàng - Nạp tiền": "PAY",
    "Nạp tiền từ Ngân hàng vào Ví": "PAY",
    "Nạp tiền điện thoại": "PAY",
    "Nạp data": "PAY",
    "Rút tiền": "PAY",
    "Rút tiền từ Ví về Ngân hàng": "PAY",
    "Rút tiền về NHLK": "PAY",
    "Chuyển tiền đến Ngân hàng": "PAY",
    "Chuyển/nhận tiền Ví - Ví": "PAY",
    "Chuyển/nhận tiền Ví - Ví Nonbank": "PAY",
    "Chuyển tiền đến Ví MoMo": "PAY",
    "Chuyển nhận tiền Ví MoMo": "PAY",
    "Chuyển/nhận tiền qua Chat": "PAY",
    "Chuyển nhận tiền qua chat": "PAY",
    "Chuyển Nhận Tiền": "PAY",
    "Kiều hối": "PAY",
    "Link nhận tiền": "PAY",
    "Nhắc trả tiền": "PAY",
    "Chia tiền": "PAY",
    "Quỹ nhóm": "PAY",

    # ===================== BILL | Bill / Utility =====================
    "Điện": "BILL",
    "Nước": "BILL",
    "Internet": "BILL",
    "Truyền hình": "BILL",
    "Truyền hình trực tuyến": "BILL",
    "Thanh toán online": "BILL",
    "Thanh toán online - Top Brand": "BILL",
    "Thanh toán trực tiếp": "BILL",
    "Thanh toán tại quầy (Offline)": "BILL",
    "Thanh toán điện thoại trả sau": "BILL",
    "Thanh toán điện thoại cố định": "BILL",
    "Thanh toán trung gian Viễn thông Tiện ích": "BILL",
    "Thu hộ bảo hiểm y tế": "BILL",
    "Thu hộ Bảo hiểm": "BILL",
    "Thu hộ thẻ tín dụng": "BILL",
    "Viện phí": "BILL",
    "Học phí": "BILL",
    "Chung cư": "BILL",
    "Dịch vụ công": "BILL",
    "Viễn thông tiện ích": "BILL",
    "Xăng dầu": "BILL",
    "Xuất hóa đơn": "BILL",
    "Dịch vụ VTTI": "BILL",
    "Dịch vụ Offline payments": "BILL",

    # ===================== MER | Merchant / Business =====================
    "QR đa năng": "MER",
    "QR Đa năng dùng MoMo quét": "MER",
    "QR Đa năng dùng App Bank/ Ví điện tử khác quét": "MER",
    "QR Bán hàng dùng MoMo quét": "MER",
    "QR Bán hàng dùng App Bank/ Ví điện tử khác quét": "MER",
    "QR bán hàng": "MER",
    "Merchant Care": "MER",
    "Doanh nghiệp/Chuỗi cửa hàng": "MER",
    "Trang doanh nghiệp": "MER",
    "EPS - ENTERPRISE PAYMENT & BUSINESS SOLUTIONS": "MER",
    "Dịch vụ W2W/W2B/Merchant Care": "MER",
    "W2B - Chuyển tiền tới ngân hàng": "MER",
    "W2B - AS - Chuyển tiền tới ngân hàng": "MER",
    "Quảng cáo": "MER",
    "Truy thu theo yêu cầu từ đối tác": "MER",
    "Quản lý thẻ thành viên": "MER",
    "Dịch vụ Marketing & Distribution": "MER",
    "Dịch vụ Digital platforms": "MER",
    "Ngân hàng - Liên kết": "MER",

    # ===================== FIN | Financial Services =====================
    "Vay Nhanh": "FIN",
    "Chi hộ vay tiêu dùng": "FIN",
    "Thu hộ vay tiêu dùng": "FIN",
    "Chi hộ vay kinh doanh": "FIN",
    "Tài chính siêu tốc": "FIN",
    "Đầu tư tích lũy": "FIN",
    "Túi thần tài": "FIN",
    "Heo đất MoMo": "FIN",
    "Chứng khoán": "FIN",
    "Chứng chỉ quỹ": "FIN",
    "Mua bảo hiểm": "FIN",
    "Thanh toán phí bảo hiểm": "FIN",
    "Tài Chính - Bảo Hiểm": "FIN",
    "Trung tâm tài chính": "FIN",
    "Quản lý chi tiêu": "FIN",
    "Vượt Ngàn Chi Tiêu": "FIN",
    "Quốc tế": "FIN",
    "Xổ số": "FIN",
    "Mở thẻ tín dụng": "FIN",
    "Tính năng tài chính": "FIN",
    "Dịch vụ FS": "FIN",

    # ===================== COM | Commerce / Lifestyle =====================
    "Thương mại điện tử": "COM",
    "Sàn thương mại điện tử": "COM",
    "Mua sắm hoàn tiền": "COM",
    "Thẻ quà tặng": "COM",
    "Mua mã thẻ di động": "COM",
    "Mua mã thẻ data": "COM",
    "Thanh toán game/ mua mã thẻ game": "COM",
    "Gói combo nhà mạng": "COM",
    "Combo Data Flexible": "COM",
    "Vé máy bay": "COM",
    "Vé tàu hỏa": "COM",
    "Vé xe khách": "COM",
    "Vé xem phim": "COM",
    "Du lịch": "COM",
    "Khách sạn": "COM",
    "Mua sim": "COM",
    "Nội dung số": "COM",
    "Giải trí": "COM",
    "Thời trang": "COM",
    "Trả góp sản phẩm Apple": "COM",
    "Vận chuyển": "COM",       
    "Vận tải": "COM",

    # ===================== PRO | Promotion / Rewards =====================
    "MoMo Rewards": "PRO",
    "MoMo Reward": "PRO",
    "Khuyến mãi hàng ngày": "PRO",
    "Chương Trình Khuyến Mãi": "PRO",
    "Chương trình khuyến mãi - Nội bộ": "PRO",
    "Giới thiệu bạn bè": "PRO",
    "Lì xì": "PRO",
    "Một Vòng MoMo – Quay 100% trúng, tổng thưởng đến 3 tỷ": "PRO",
    "Vòng Quay \"Chill\" Tài": "PRO",
    "Vòng Phát Tài": "PRO",
    "Đấu Trường Tri Thức": "PRO",
    "Sống tốt cùng MoMo": "PRO",
    "Thổ địa MoMo": "PRO",
    "Đi bộ cùng MoMo": "PRO",
    "Tích tem dễ dàng, săn ngàn ưu đãi từ thương hiệu Nhật và quà Conan độc quyền": "PRO",

    # ===================== APP | App / Platform =====================
    "Hướng dẫn sử dụng ứng dụng": "APP",
    "Các tính năng trên MoMo": "APP",
    "Tính năng": "APP",
    "Tính Năng": "APP",
    "Mini Apps": "APP",
    "Báo lỗi ứng dụng": "APP",
    "Lỗi liên quan nền tảng": "APP",
    "Nền tảng ứng dụng": "APP",
    "Màn hình Kết quả giao dịch": "APP",
    "Màn hình thanh toán": "APP",
    "Màn hình thanh toán an toàn": "APP",
    "Trò chuyện trên ứng dụng": "APP",
    "Trang cá nhân trên ứng dụng": "APP",
    "Hộ chiếu sinh viên": "APP",
    "XNA": "APP",
    "Môi trường": "APP",
    "Đối tác MoMo": "APP",
    "Điểm hỗ trợ MoMo": "APP",

    # ===================== CS | Customer Support =====================
    "CSKH": "CS",
    "Giải quyết khiếu nại": "CS",
    "Phản ánh lừa đảo": "CS",
    "Góp ý": "CS",
    "Góp ý chưa rõ ràng": "CS",
 

    # ===================== OTHER | Long-tail =====================
    "Khác": "OTHER",
    "Dịch vụ khác": "OTHER",
    "Chưa xác định được vấn đề": "Undefined"
}



df_data_ticket["issue_group_code"] = (
    df_data_ticket["Level1"]
    .map(issue_group_map)
)

issue_group_label_map = {
    "ACC": "Account & Security",
    "PAY": "Payments & Transfer",
    "BILL": "Bill & Utilities",
    "MER": "Merchant & Business",
    "FIN": "Financial Services",
    "COM": "Commerce & Lifestyle",
    "PRO": "Promotion & Rewards",
    "APP": "App & Platform",
    "CS": "Customer Support",
    "OTHER": "Other",
    "Undefined": "Undefined Problems"
}

df_data_ticket["issue_group_label"] = (
    df_data_ticket["issue_group_code"]
    .map(issue_group_label_map)
)


In [451]:
count_per_group=df_data_ticket.groupby('issue_group_label').agg({'Total_ticket':'sum'}).reset_index().sort_values('Total_ticket', ascending=False)

In [452]:
count_per_group

,issue_group_label,Total_ticket
5,Financial Services,53978
10,Undefined Problems,47910
0,Account & Security,18409
8,Payments & Transfer,14422
3,Commerce & Lifestyle,13260
6,Merchant & Business,12304
2,Bill & Utilities,8361
1,App & Platform,7203
4,Customer Support,1719
9,Promotion & Rewards,1634


In [453]:
count_per_issue=df_data_ticket.groupby(['issue_group_label','Level1']).agg({'Total_ticket':'sum'}).reset_index().sort_values('Total_ticket', ascending=False)

In [454]:
count_per_issue.head(10)

,issue_group_label,Level1,Total_ticket
170,Undefined Problems,Chưa xác định được vấn đề,47910
93,Financial Services,Chi hộ vay tiêu dùng,28785
101,Financial Services,Quốc tế,14150
4,Account & Security,Hỗ trợ tài khoản,9764
103,Financial Services,Thu hộ vay tiêu dùng,5709
131,Merchant & Business,W2B - Chuyển tiền tới ngân hàng,5306
148,Payments & Transfer,Nạp tiền,4867
26,App & Platform,Hướng dẫn sử dụng ứng dụng,4565
130,Merchant & Business,W2B - AS - Chuyển tiền tới ngân hàng,3423
112,Financial Services,Đầu tư tích lũy,2759


# Part B

In [215]:
df_working["EVENT_TIME"] = pd.to_datetime(df_working["EVENT_TIME"])
df_working["WEEK_START_DATE"] = pd.to_datetime(df_working["WEEK_START_DATE"])

# Actual working day
df_working["DATE"] = df_working["EVENT_TIME"].dt.date

# Aggregate working hours per agent per day per week
working_daily = (
    df_working
    .groupby(
        ["AGENT_ID", "YEAR", "MONTH", "WEEK", "WEEK_START_DATE", "DATE"],
        as_index=False
    )
    .agg(
        TOTAL_WORKING_HOURS=("DURATION_HOURS", "sum")
    )
)

df_data_ticket["Date"] = pd.to_datetime(df_data_ticket["Date"]).dt.date

ticket_daily = (
    df_data_ticket
    .groupby(
        ["Agent_tiep_nhan", "Year", "Month", "Week", "Date"],
        as_index=False
    )
    .agg(
        TOTAL_TICKET=("Total_ticket", "sum")
    )
)

ticket_daily = ticket_daily.rename(columns={
    "Agent_tiep_nhan": "AGENT_ID",
    "Year": "YEAR",
    "Month": "MONTH",
    "Week": "WEEK",
    "Date": "DATE"
})



final_df = working_daily.merge(
    ticket_daily,
    on=["AGENT_ID", "YEAR", "MONTH", "WEEK", "DATE"],
    how="outer"
)

final_df["TOTAL_TICKET"] = final_df["TOTAL_TICKET"].fillna(0)

final_df["DATE"] = pd.to_datetime(final_df["DATE"])

final_df["WEEK_START_DATE"] = (
    final_df["DATE"]
    - pd.to_timedelta(final_df["DATE"].dt.weekday, unit="D")
)

final_df["WEEK_END_DATE"] = final_df["WEEK_START_DATE"] + pd.Timedelta(days=6)

final_df["WEEK_RANGE"] = (
    final_df["WEEK_START_DATE"].dt.strftime("%Y-%m-%d")
    + " to "
    + final_df["WEEK_END_DATE"].dt.strftime("%Y-%m-%d")
)

overview_date = final_df[[
    "YEAR",
    "MONTH",
    "WEEK",
    "WEEK_RANGE",
    "DATE",                # actual day happen
    "AGENT_ID",
    "TOTAL_WORKING_HOURS",
    "TOTAL_TICKET"
]].sort_values(["DATE", "AGENT_ID"])





In [216]:
overview_date.head()

,YEAR,MONTH,WEEK,WEEK_RANGE,DATE,AGENT_ID,TOTAL_WORKING_HOURS,TOTAL_TICKET
2,2025,6,22,2025-05-26 to 2025-06-01,2025-06-01,bpomb02,0.588333,31.0
5,2025,6,22,2025-05-26 to 2025-06-01,2025-06-01,bpomb09,10.201667,90.0
110,2025,6,22,2025-05-26 to 2025-06-01,2025-06-01,bpomb13,6.929722,88.0
163,2025,6,22,2025-05-26 to 2025-06-01,2025-06-01,bpomb15,7.399444,72.0
222,2025,6,22,2025-05-26 to 2025-06-01,2025-06-01,bpomb16,0.544722,24.0


In [217]:
# 1. Aggregate về level AGENT_ID × WEEK (fact chuẩn)
agent_week_df = (
    final_df
    .groupby(
        ["AGENT_ID", "YEAR", "WEEK", "WEEK_RANGE"],
        as_index=False
    )
    .agg(
        TOTAL_WORKING_HOURS=("TOTAL_WORKING_HOURS", "sum"),
        TOTAL_TICKET=("TOTAL_TICKET", "sum"),
    )
)

# 2. Roll up về level WEEK (weekly overview)
week_df = (
    agent_week_df
    .groupby(
        ["YEAR", "WEEK", "WEEK_RANGE"],
        as_index=False
    )
    .agg(
        TOTAL_AGENT=("AGENT_ID", "nunique"),
        TOTAL_TICKET=("TOTAL_TICKET", "sum"),
        TOTAL_WORKING_HOURS=("TOTAL_WORKING_HOURS", "sum"),
    )
)

# 3. Tạo YEAR_WEEK_KEY (để join an toàn qua năm)
week_df["YEAR_WEEK_KEY"] = week_df["YEAR"] * 100 + week_df["WEEK"]

# 4. Tạo self-table cho LAST WEEK
last_week_df = week_df[
    [
        "YEAR_WEEK_KEY",
        "TOTAL_AGENT",
        "TOTAL_TICKET",
        "TOTAL_WORKING_HOURS",
    ]
].rename(
    columns={
        "YEAR_WEEK_KEY": "LAST_YEAR_WEEK_KEY",
        "TOTAL_AGENT": "TOTAL_AGENT_LAST_WEEK",
        "TOTAL_TICKET": "TOTAL_TICKET_LAST_WEEK",
        "TOTAL_WORKING_HOURS": "TOTAL_WORKING_HOURS_LAST_WEEK",
    }
)

# 5. Tạo join key = last week + 1
last_week_df["JOIN_KEY"] = last_week_df["LAST_YEAR_WEEK_KEY"] + 1

# 6. Self-join để lấy metric tuần trước (chuẩn nghĩa)
overview_week = week_df.merge(
    last_week_df[
        [
            "JOIN_KEY",
            "TOTAL_AGENT_LAST_WEEK",
            "TOTAL_TICKET_LAST_WEEK",
            "TOTAL_WORKING_HOURS_LAST_WEEK",
        ]
    ],
    how="left",
    left_on="YEAR_WEEK_KEY",
    right_on="JOIN_KEY",
)

# 7. Clean output để dùng cho Looker
overview_week = overview_week.drop(
    columns=["YEAR_WEEK_KEY", "JOIN_KEY"]
)

overview_week["AVG_TICKET_PER_HOUR"] = np.where(
    overview_week["TOTAL_WORKING_HOURS"] > 0,
    overview_week["TOTAL_TICKET"] / overview_week["TOTAL_WORKING_HOURS"],
    np.nan
)

overview_week["AVG_WH_PER_EMPLOYEE"] = np.where(
    overview_week["TOTAL_AGENT"] > 0,
    overview_week["TOTAL_WORKING_HOURS"] / overview_week["TOTAL_AGENT"],
    np.nan
)

overview_week["AVG_TICKET_PER_EMPLOYEE"] = np.where(
    overview_week["TOTAL_AGENT"] > 0,
    overview_week["TOTAL_TICKET"] / overview_week["TOTAL_AGENT"],
    np.nan
)


# ===== LAST WEEK =====

overview_week["AVG_TICKET_PER_HOUR_LAST_WEEK"] = np.where(
    overview_week["TOTAL_WORKING_HOURS_LAST_WEEK"] > 0,
    overview_week["TOTAL_TICKET_LAST_WEEK"]
    / overview_week["TOTAL_WORKING_HOURS_LAST_WEEK"],
    np.nan
)

overview_week["AVG_WH_PER_EMPLOYEE_LAST_WEEK"] = np.where(
    overview_week["TOTAL_AGENT_LAST_WEEK"] > 0,
    overview_week["TOTAL_WORKING_HOURS_LAST_WEEK"]
    / overview_week["TOTAL_AGENT_LAST_WEEK"],
    np.nan
)

overview_week["AVG_TICKET_PER_EMPLOYEE_LAST_WEEK"] = np.where(
    overview_week["TOTAL_AGENT_LAST_WEEK"] > 0,
    overview_week["TOTAL_TICKET_LAST_WEEK"]
    / overview_week["TOTAL_AGENT_LAST_WEEK"],
    np.nan
)



In [218]:
overview_date.to_csv('overview_date.csv',index=False)
overview_week.to_csv('overview_week.csv',index=False)
df_data_ticket.to_csv('df_data_ticket.csv',index=False)


# Part E

In [233]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)   # remove special characters
    text = re.sub(r"\s+", " ", text)       # remove extra spaces
    return text.strip()


In [234]:
df_customer_feedback["problem_text"] = df_customer_feedback["Customer problems"].apply(clean_text)
df_customer_feedback["comment_text"] = df_customer_feedback["Customer comments after the ticket is closed"].apply(clean_text)


In [315]:
problem_categories = {

    # 1. Missing / deducted money
    "Money missing / Deducted but not received": [
        "không nhận được tiền", "chưa nhận được tiền", "vẫn chưa nhận",
        "tiền chưa về", "tiền chưa vào", "chưa vào tiền",
        "không thấy tiền", "không vào ví", "chưa có thông báo nhận tiền",
        "bị trừ tiền", "trừ tiền nhưng", "trừ tiền không rõ",
        "tiền trong ví không hợp lý", "mất tiền", "thiếu tiền", "mắt tiền",
        "bị thiếu hụt tiền", "số dư không đúng",
        "tiền bị giữ", "giữ tiền", "bị giam tiền",
        "không nhận đủ tiền", "tiền đâu", "bị trừ", "trừ tiền"
        ,"chưa vào tien"
    ],

    # 2. Transfer issues
    "Transfer issues / Recipient not received": [
        "chuyển tiền", "chuyển khoản", "ck", "ck thành công",
        "chuyển thành công", "chuyển không được",
        "không chuyển được", "chuyển tiền không thành công",
        "người nhận chưa nhận", "bên kia chưa nhận",
        "bên nhận chưa có", "bên nhận không nhận được",
        "treo tiền bên nhận",
        "chuyển nhầm", "chuyển sai", "ghi nhầm", "chuyển lộn",
        "qr sai", "quét mã", "không quét được", "nạp tiền"
    ],

    # 3. Refund issues
    "Refund not processed": [
        "hoàn tiền", "chưa hoàn tiền", "chưa được hoàn",
        "tiền hoàn", "hoàn trả", "chưa thấy hoàn",
        "bao giờ hoàn", "khi nào hoàn", "chờ hoàn",
        "muốn hoàn lại tiền", "hoàn lại", "tiền chưa hoàn về",
        "chưa trả lại tiền"
    ],

    # 4. Transaction pending / held
    "Transaction pending / Held": [
        "đang xử lý", "giao dịch treo", "pending",
        "tiền bị giữ", "đợi xử lý", "chờ xử lý",
        "xử lý quá lâu", "xử lý lâu",
        "chưa có phản hồi", "treo từ",
        "đợi hoài", "đợi mãi", "treo nhiều ngày"
    ],

    # 5. Payment failed but charged
    "Failed transaction but charged": [
        "giao dịch thất bại", "thanh toán không thành công",
        "không thành công nhưng vẫn trừ",
        "báo lỗi nhưng trừ tiền", "báo thất bại nhưng trừ tiền",
        "app báo lỗi", "app báo lỗi nhưng trừ",
        "nạp thất bại", "nạp xong báo lỗi",
        "lỗi nhưng trừ", "giao dịch lỗi nhưng trừ"
    ],

    # 6. Duplicate / double charge
    "Duplicate / Double charge": [
        "trừ 2 lần", "thanh toán 2 lần", "bị trừ hai lần",
        "trừ gấp đôi", "thanh toán thêm lần nữa",
        "trừ thêm", "bị trừ lại",
        "đóng thừa", "trừ dư", "thanh toán thừa"
    ],

    # 7. Auto charge / Unauthorized payment
    "Unauthorized / Auto charge": [
        "tự động trừ tiền", "tự trừ", "tự ý trừ",
        "không đồng ý thanh toán", "không được sự đồng ý",
        "chưa xác nhận", "tự ý thanh toán",
        "gia hạn tự động", "không đăng ký",
        "không thực hiện giao dịch", "tôi không mua",
        "tôi không thanh toán", "không hề mua",
        "không có nhu cầu"
    ],

    # 8. Product / Service not delivered
    "Paid but service not delivered": [
        "không nhận được sản phẩm", "thanh toán thành công nhưng",
        "không có data", "không dùng được",
        "game không nhận", "game không cộng",
        "nạp game không nhận",
        "vé không nhận", "không nhận được vé",
        "không có mã", "mã không cộng",
        "gói không hoạt động", "không sử dụng được",
        "không có số ghế", "không nhận được sản phẩm trong game"
    ],

    # 9. Subscription / Cancellation
    "Subscription / Cancellation issues": [
        "huỷ giao dịch", "huỷ gói", "huỷ dịch vụ",
        "huỷ yêu cầu", "thu hồi giao dịch", "muốn thu hồi",
        "không muốn gia hạn", "tự gia hạn",
        "dùng thử", "huỷ thanh toán", "không muốn dùng nữa"
    ],

    # 10. Financial services (loan / BNPL / debt)
    "Loan / BNPL / Debt issues": [
        "ví trả sau", "ví trả sao", "mo vi tra sau",
        "khoản vay", "vay", "trả góp",
        "dư nợ", "nhắc nợ", "bị gọi đòi nợ", "fe gọi",
        "tất toán", "hạn mức",
        "điểm tín dụng", "lãi", "quá hạn"
    ],

    # 11. Bank linking / sync
    "Bank linking / Bank sync issues": [
        "liên kết ngân hàng", "không liên kết được",
        "ngân hàng chưa nhận", "ngân hàng báo lỗi",
        "ngân hàng đã xác nhận", "ngân hàng nói không lỗi",
        "bên ngân hàng treo tiền",
        "phát sinh phí", "otp ngân hàng", "otp báo lỗi",
        "vietcom", "vcb", "bidv", "tpbank", "vib", "mb", "acb",
        "vpbank", "agribank"
    ],

    # 12. Account / Identity / Security
    "Account / Identity issues": [
        "tài khoản", "khóa ví", "mở khoá",
        "xoá ví", "huỷ ví",
        "cccd", "sinh trắc", "khuôn mặt",
        "xác thực", "xác thực lỗi", "xác thực rồi mà lỗi",
        "otp lỗi", "mất sim", "đổi số điện thoại",
        "không giao dịch được", "duyệt lâu",
        "chưa được duyệt", "đang dùng 3 tk momo"
    ],

    # 13. Promotion / Reward / Point
    "Promotion / Reward / Point issues": [
        "voucher", "khuyến mãi", "xu",
        "quà", "momo rewards", "bước chân",
        "điểm", "điểm tin cậy",
        "mã ưu đãi", "không áp được mã",
        "vòng quay", "quay không được",
        "tokens", "không nhận được tokens",
        "không nhận được mã mời", "không tham gia được"
    ],

    # 14. Partner platform issues
    "Partner platform issues": [
        "grab", "xanhsm", "google", "tiktok",
        "netflix", "facebook", "lazada",
        "roblox", "vng", "apple", "app store",
        "zalopay", "tv360", "canva",
        "đối tác", "nền tảng", "game","xem phim"
    ],

    # 15. Fraud / Scam
    "Fraud / Scam": [
        "lừa đảo", "bị lừa", "bị lừa đảo",
        "chiếm đoạt", "scam",
        "lừa tiền", "gian lận", "nghi lừa đảo"
    ],

    # 16. Customer service quality
    "Customer service quality": [
        "cskh", "chăm sóc khách hàng",
        "phản hồi chậm", "không phản hồi",
        "trả lời vòng vo", "không giải thích rõ",
        "thái độ", "máy móc",
        "không hỗ trợ", "không xử lý",
        "không ai giải quyết", "hỗ trợ kém"
    ],

    # 17. VietJet / Flight / Ticket
    "Flight / VietJet ticket issues": [
        "vietjet", "vé máy bay", "vé bay",
        "chuyến bay", "delay", "hoãn chuyến",
        "hành lý", "ký gửi", "đổi vé",
        "huỷ vé", "bảo lưu", "boarding",
        "mã đặt chỗ", "không xem được vé",
        "không nhận được vé", "không có số ghế"
    ],

    # 18. Card / Top-up / Voucher code
    "Card / Top-up / Voucher issues": [
        "thẻ", "mã thẻ", "card", "voucher",
        "không nhận được mã", "không nhập được mã",
        "mã không dùng được", "thẻ đã sử dụng",
        "nạp thẻ không được", "thẻ lỗi",
        "cấp lại mã thẻ"
    ],

    # 19. Mobile Data / Telco
    "Mobile data / Telco package issues": [
        "data", "3g", "4g", "5g",
        "gói data", "gói cước",
        "không có dung lượng", "mất dung lượng",
        "không sử dụng được", "không kích hoạt được",
        "viettel", "mobifone", "vinaphone"
    ]
}


In [316]:
def assign_problem_category(text):
    for category, keywords in problem_categories.items():
        for kw in keywords:
            if kw in text:
                return category
    return "Other / Unclear"

df_customer_feedback["problem_category"] = df_customer_feedback["problem_text"].apply(assign_problem_category)

In [317]:
def assign_problem_category(text):
    for category, keywords in problem_categories.items():
        for kw in keywords:
            if kw in text:
                return category
    return "Other / Unclear"

df_customer_feedback["problem_category"] = df_customer_feedback["problem_text"].apply(assign_problem_category)

In [425]:
comment_categories = {

    "Angry / Escalation": [
        "địt", "đm", "đéo", "như cc", "lừa đảo",
        "scam", "vô dụng", "vote 1 sao", "xóa app",
        "mất tiền oan", "làm ăn như",     "như con cặc",
    "như cc",
    "đầu c",
    "bố láo",
    "lừa",
    "lừa tiền",
    "cướp tiền",
    "ăn cướp",
    "phốt",
    "đéo",
    "dm",
    "đm",
    "kiện",
    "công an",
    "vote app",
    "xóa app",
    "gỡ app",
    "tệ",
        "noi lao",
    "bip ha",
    "bịp",
    "lua",
    "cuop tien",
    "khong dang dung",
    "xoa app",
    "ai dam de tien",
    "qua te"
    ],

    "Unresolved / Not solved": [
        "chưa giải quyết", "chưa được giải quyết",
        "chưa xử lý", "chưa xong",
        "chưa hoàn tiền", "tiền chưa về",
        "chưa nhận được tiền", "vẫn chưa nhận",
        "người nhận chưa nhận", "bên kia chưa nhận",
        "chuyển khoản thành công nhưng",
        "chưa thấy tiền", "chưa hỗ trợ",
        "không giải quyết", "không xử lý",
            # viết tắt / sai chính tả
        "chx", "ch", "van chua", "vân chưa", "kk nhan", "k nhận",
         "chưa dc", "chưa đc", "chưa dk", "chưa nhạn",
    
        # hoàn tiền / tiền
        "chưa hồi tiền", "chưa thấy hoàn", "chưa trả tiền",
        "tiền chưa cộng", "tiền chưa vô",
        "chưa update tiền", "chưa cập nhật tiền",
        
        # xử lý
        "chưa giải viết", "chưa sử lí", "chưa xử lí",
        "vẫn để thanh toán",
        "chưa có kết quả",
        
        # chuyển khoản
        "kk nhận dc", "ng nhận chưa",
        "bên kia chưa nhận",
        "chuyển tiền nhưng",
        "ck thành công nhưng",
            "khong hoan tien",
    "da xu ly nhung",
    "da xu ly ma",
    "khong thay tien",
    "khong co tien",
    "tien bi tru",
    "tien khong ve",
    "nguoi nhan khong nhan",
    "ben kia khong nhan",
    "treo tien",
    "giu tien",
    "van khong duoc",
    "khong co ket qua",
    "van de van con",
    "tien cua toi dau",
        "qua lau",
    "rat lau",
    "khong thoa dang",
    "khong lien quan",
    "so du vo ly",
    "mat tien",
    "may tra loi",
    "khong doc noi dung",
    "xu ly qua lau",
    "phan hoi cham"
    ],

    "Pending / Waiting": [
        "đang chờ", "vẫn đang chờ",
        "chờ xử lý", "đợi xử lý",
        "đang xử lý", "vẫn đang xử lý",
        "qua 48h", "bao giờ", "khi nào","đợi phản hồi",
    "chờ phản hồi",
    "chờ xử lý",
    "đang trong hàng chờ",
    "đang treo",
    "đang bị giữ",
    "qua giờ chưa",
    "đến nay vẫn chưa",
    "hơn 2 ngày",
    "hơn 1 tuần",
    "đợi mãi",
    "vẫn phải đợi",
        "hay giai quyet",
    "vui long lien he",
    "mong giai quyet",
    "can giai quyet",
    "can xu ly",
    "dang doi",
    "cho xu ly",
    "cho phan hoi",
    "da cung cap thong tin",
    "mong giải quyết sớm",
    "xử lý nhanh"
    ],

    "Dissatisfied / Complaint": [
        "quá chậm", "rất lâu", "chờ quá lâu", "lâu", 
        "phản hồi chậm", "vòng vo",
        "trả lời không đúng trọng tâm",
        "không rõ ràng", "không thỏa đáng",
        "dịch vụ kém", "không hài lòng",
        "bức xúc", "thất vọng", 
        "tệ",
        "quá tệ",
        "chán",
        "rất chán",
        "bực",
        "bực mình",
        "khó chịu",
        "vô lý",
        "không hợp lý",
        "thiếu trách nhiệm",
        "máy móc",
        "nói chung chung",
        "trả lời cho có",
        "đùn đẩy",
        "đổ lỗi",
        "không đọc nội dung",
        "không đúng trọng tâm",
        
    ],

    "Resolved but slow": [
        "cuối cùng cũng giải quyết",
        "xử lý hơi lâu",
        "hơi lâu mà",
        "tuy trễ nhưng",
        "lâu nhưng đã xong"
    ],

    "Resolved / Very satisfied": [
        "đã giải quyết", "đã xử lý xong",
        "đã hoàn tiền", "đã nhận được tiền",
        "đã ổn", "đã giải quyết rồi",
        "rất hài lòng", "tuyệt vời",
        "rất tốt", "hỗ trợ tốt",
        "nhân viên nhiệt tình",
        "excellent", "very good", "good",     "hài long", "hai long",
    "tuyet voi",
    "rat hai long",
    "qua hai long",
    "10 10 sao",
    "dv tot",
    "rat nhiet tinh",
    "uy tin",
    "da giai quyet roi",
    "da xu ly xong"
    ],

    "Neutral / Acknowledged": [
        "ok", "oke", "vâng", "dạ",
        "đồng ý", "đã hiểu",
        "cảm ơn", "cám ơn",
        "thanks", "thank you", "tks"
    ]
}


In [426]:
CATEGORY_PRIORITY = [
    "Angry / Escalation",
    "Unresolved / Not solved",
    "Pending / Waiting",
    "Dissatisfied / Complaint",
    "Resolved but slow",
    "Resolved / Very satisfied",
    "Neutral / Acknowledged"
]



def assign_category(text):
    for category in CATEGORY_PRIORITY:
        keywords = comment_categories.get(category, [])
        for kw in keywords:
            if kw in text:
                return category
    return "Other"


df_customer_feedback["comment_category"] = df_customer_feedback["comment_text"].apply(assign_category)



In [428]:
df_customer_feedback.to_csv('df_customer_feedback.csv', index=False)

In [440]:
problem_dist = (
    df_customer_feedback['problem_category']
    .value_counts()
    .reset_index()
    .rename(columns={
        'number_of_cases': 'problem_category',
        'count': 'number_of_cases'
    })
)


problem_dist['percentage (%)'] = round((
    problem_dist['number_of_cases'] / problem_dist['number_of_cases'].sum()
)*100,2)

problem_dist


,problem_category,number_of_cases,percentage (%)
0,Other / Unclear,259,25.93
1,Money missing / Deducted but not received,227,22.72
2,Transfer issues / Recipient not received,102,10.21
3,Refund not processed,100,10.01
4,Loan / BNPL / Debt issues,89,8.91
5,Account / Identity issues,64,6.41
6,Promotion / Reward / Point issues,37,3.70
7,Bank linking / Bank sync issues,21,2.10
8,Partner platform issues,20,2.00
9,Card / Top-up / Voucher issues,12,1.20
